# **Yelp API**

## **Imports**

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

## **Calling API**

In [2]:
# Load API Credentials
with open('/Users/davyd/.secret/yelp_api.json','r') as f:
    login = json.load(f)

In [3]:
# Login Key
login.keys()

dict_keys(['client-id', 'api-key'])

In [4]:
# Instantiate YelpAPI Variable
yelp = YelpAPI(login['api-key'],timeout_s=5.0)

## **Defining Search Parameters**

In [5]:
# set our API call parameters and filename before the first call
location = 'Brooklyn, NY 11211'
term = 'Pizza'

In [6]:
location.split(',')[0]

'Brooklyn'

In [7]:
## Specify fodler for saving data


FOLDER ='Data/'

os.makedirs(FOLDER,exist_ok=True)

# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER+f"{location.split(',')[0]}-{term}.json"

In [8]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)

## If it does not exist: 
if file_exists ==False:
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE,'w')as f:
        json.dump([],f)
        
## If it exists, inform user
else:
       print(f"[i] {JSON_FILE} already exists.")

[i] Data/Brooklyn-Pizza.json already exists.


In [9]:
results = yelp.search_query(term=term,location = location)

In [10]:
type(results)

dict

In [11]:
len(results)

3

In [12]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [13]:
results['total']

5500

In [14]:
results['region']

{'center': {'longitude': -73.94966125488281, 'latitude': 40.71264321986641}}

In [15]:
results['businesses']

[{'id': 'v1DHGRNCH9247WLYoaoA9A',
  'alias': 'l-industrie-pizzeria-brooklyn',
  'name': "L'industrie Pizzeria",
  'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/1-VBwAOj0g9YKYey3V-Z6Q/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/l-industrie-pizzeria-brooklyn?adjust_creative=TqzhtA42spPSKOmzDuw9JA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=TqzhtA42spPSKOmzDuw9JA',
  'review_count': 1160,
  'categories': [{'alias': 'pizza', 'title': 'Pizza'}],
  'rating': 4.5,
  'coordinates': {'latitude': 40.71162, 'longitude': -73.95783},
  'transactions': ['delivery'],
  'price': '$',
  'location': {'address1': '254 S 2nd St',
   'address2': '',
   'address3': '',
   'city': 'Brooklyn',
   'zip_code': '11211',
   'country': 'US',
   'state': 'NY',
   'display_address': ['254 S 2nd St', 'Brooklyn, NY 11211']},
  'phone': '+17185990002',
  'display_phone': '(718) 599-0002',
  'distance': 701.8952662330755},
 {'id': 'ON7ihGoWgZaBFDaLCYpk2Q',
  'alias'

In [16]:
pd.DataFrame(results['businesses'])

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,v1DHGRNCH9247WLYoaoA9A,l-industrie-pizzeria-brooklyn,L'industrie Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/1-VBwA...,False,https://www.yelp.com/biz/l-industrie-pizzeria-...,1160,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.71162, 'longitude': -73.95783}",[delivery],$,"{'address1': '254 S 2nd St', 'address2': '', '...",+17185990002,(718) 599-0002,701.895266
1,ON7ihGoWgZaBFDaLCYpk2Q,rome-to-brooklyn-pizza-brooklyn-2,Rome To Brooklyn Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/4JavGL...,False,https://www.yelp.com/biz/rome-to-brooklyn-pizz...,166,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,"{'latitude': 40.711886016839244, 'longitude': ...","[pickup, delivery]",NaN,"{'address1': '755 Grand St', 'address2': None,...",+17182693239,(718) 269-3239,576.009202
2,zj8Lq1T8KIC5zwFief15jg,prince-street-pizza-new-york-2,Prince Street Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/PfI8oV...,False,https://www.yelp.com/biz/prince-street-pizza-n...,5092,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.72308755605564, 'longitude': -...","[pickup, delivery]",$,"{'address1': '27 Prince St', 'address2': None,...",+12129664100,(212) 966-4100,3955.767847
3,r1PWMTbFXBmEhi0scGPf3w,aces-pizza-brooklyn,Ace's Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/id-UXj...,False,https://www.yelp.com/biz/aces-pizza-brooklyn?a...,172,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.71505, 'longitude': -73.958365}",[delivery],$$,"{'address1': '637 Driggs Ave', 'address2': '',...",+13477254366,(347) 725-4366,782.179586
4,GRXvvB1OKMuW4XDWEaVAlg,joes-pizza-brooklyn-8,Joe's Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/0WEXB_...,False,https://www.yelp.com/biz/joes-pizza-brooklyn-8...,459,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.7168952, 'longitude': -73.9589...","[pickup, delivery]",$,"{'address1': '216 Bedford Ave', 'address2': ''...",+17183882216,(718) 388-2216,913.119726
5,CFR-HzVHKWRMpMAR3LAhzg,montesacro-pinseria-brooklyn-brooklyn,Montesacro Pinseria Brooklyn,https://s3-media2.fl.yelpcdn.com/bphoto/5xTDQs...,False,https://www.yelp.com/biz/montesacro-pinseria-b...,266,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.71338, 'longitude': -73.95136}","[pickup, delivery]",$$,"{'address1': '432 Union Ave', 'address2': None...",+13479161062,(347) 916-1062,149.087879
6,WE_dce2JKcv-VoFNaqiJ4w,carmines-pizzeria-brooklyn,Carmine's Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/V6LFN2...,False,https://www.yelp.com/biz/carmines-pizzeria-bro...,538,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 40.714987, 'longitude': -73.94414}","[pickup, delivery]",$$,"{'address1': '358 Graham Ave', 'address2': '',...",+17187829659,(718) 782-9659,530.760498
7,FhXjAc6nKLf414KxYujUHw,robertas-brooklyn-2,Roberta's,https://s3-media2.fl.yelpcdn.com/bphoto/Cx9X4Q...,False,https://www.yelp.com/biz/robertas-brooklyn-2?a...,3322,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 40.70493, 'longitude': -73.93399}","[pickup, delivery]",$$,"{'address1': '261 Moore St', 'address2': '', '...",+17184171118,(718) 417-1118,1574.938853
8,2FgUndzCT_jJATbmrBDVBg,paulie-gees-slice-shop-brooklyn,Paulie Gee's Slice Shop,https://s3-media4.fl.yelpcdn.com/bphoto/x01yyT...,False,https://www.yelp.com/biz/paulie-gees-slice-sho...,361,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 40.72836, 'longitude': -73.9572}",[delivery],$,"{'address1': '110 Franklin St', 'address2': ''...",+19293376384,(929) 337-6384,1859.244054
9,ysqgdbSrezXgVwER2kQWKA,julianas-brooklyn-3,Juliana's,https://s3-media2.fl.yelpcdn.com/bphoto/od36nF...,False,https://www.yelp.com/biz/julianas-brooklyn-3?a...,2711,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.70274718768062, 'longitude': -...",[delivery],$$,"{'a

In [17]:
# How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [18]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total'])/ results_per_page)
n_pages

275

## **Data Cleaning**

In [19]:
# Converting the JSON file to a dataframe
df = pd.read_json(JSON_FILE)

In [20]:
# Looking at the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 980 entries, 0 to 979
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             980 non-null    object 
 1   alias          980 non-null    object 
 2   name           980 non-null    object 
 3   image_url      980 non-null    object 
 4   is_closed      980 non-null    bool   
 5   url            980 non-null    object 
 6   review_count   980 non-null    int64  
 7   categories     980 non-null    object 
 8   rating         980 non-null    float64
 9   coordinates    980 non-null    object 
 10  transactions   980 non-null    object 
 11  price          724 non-null    object 
 12  location       980 non-null    object 
 13  phone          980 non-null    object 
 14  display_phone  980 non-null    object 
 15  distance       980 non-null    float64
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 115.9+ KB


In [21]:
# Dropping irrelevant columns that do not affect predicting the coorelation or good inspection scores to good yelp ratings
df.drop(['display_phone', 'phone', 'distance', 'price', 'id', 'is_closed'], axis=1, inplace=True)

In [22]:
# Function to filter out restaurants with 20 or fewer reviews
def filter_restaurants(df):
    filtered_dataframe = df[df['review_count'] >= 20]
    return filtered_dataframe

# Example usage:
# Assuming 'your_data' is your DataFrame containing Yelp reviews
df = filter_restaurants(df)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 847 entries, 0 to 979
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   alias         847 non-null    object 
 1   name          847 non-null    object 
 2   image_url     847 non-null    object 
 3   url           847 non-null    object 
 4   review_count  847 non-null    int64  
 5   categories    847 non-null    object 
 6   rating        847 non-null    float64
 7   coordinates   847 non-null    object 
 8   transactions  847 non-null    object 
 9   location      847 non-null    object 
dtypes: float64(1), int64(1), object(8)
memory usage: 72.8+ KB


In [24]:
# Saving dataset to upload into other notebooks
df.to_csv('ydf', index=False)

In [25]:
for i in tqdm_notebook( range(1,n_pages+1)):
   # The block of code we want to TRY to run
    try:
        
        time.sleep(.2)
        
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
## save number of results for to use as offset
        n_results = len(previous_results)
        
        
        ## use n_results as the OFFSET 
        results = yelp.search_query(location=location,
                                        term=term, 
                                        offset=n_results+1)
## append new results and save to file
        previous_results.extend(results['businesses'])

        with open(JSON_FILE,'w') as f:
            json.dump(previous_results,f)
#What to do if we get an error/exception.
    except Exception as e: # saving the error message so we can print it.
        print('[!] ERROR: ',e)

  0%|          | 0/275 [00:00<?, ?it/s]

[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: To

[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: To

[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
[!] ERROR:  VALIDATION_ERROR: To